#12.1 Your First Agent

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def multiply(a, b):
    return a * b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(
            func=multiply,
            name="Multiplication Calculator",
            description="Use this to perform multiplications of two numbers. This tool take two arguments, both  should be numbers.",
        ),
    ],
)

prompt = "Cost of $2.4 x $45.3 x $23.7"
# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

llm.invoke(prompt)
# agent.invoke(prompt)

AIMessage(content='To find the cost of $2.4 multiplied by $45.3 multiplied by $23.7, you would simply multiply the three numbers together:\n\n$2.4 x $45.3 x $23.7 = $2.4 x 45.3 x 23.7 = $258.48 x 23.7 = $6130.776\n\nTherefore, the cost of $2.4 multiplied by $45.3 multiplied by $23.7 is $6130.776.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 23, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--c851ab21-b463-4b5c-83d5-d7a2b7451c16-0')

In [3]:
agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought:
To find the cost of $2.4 multiplied by $45.3 and then multiplied by $23.7, I can use the Multiplication Calculator tool.

Action:
```
{
  "action": "Multiplication Calculator",
  "action_input": {
    "a": 2.4,
    "b": 45.3
  }
}
```
Observation: 108.71999999999998
Thought:Now that we have the result of the first multiplication, we can proceed to multiply it by $23.7 to find the final cost.
Action: 
```
{
  "action": "Multiplication Calculator",
  "action_input": {
    "a": 108.72,
    "b": 23.7
  }
}
```
Observation: 2576.6639999999998
Thought:Now that we have the final result of multiplying $2.4 by $45.3 and then by $23.7, we can provide the total cost.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The cost is $2576.66"
}
```

> Finished chain.


{'input': 'Cost of $2.4 x $45.3 x $23.7', 'output': 'The cost is $2576.66'}

#12.3 Zero-shot ReAct Agent

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def multiply(inputs):
    a, b = inputs.split(",")
    return float(a) * float(b)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        Tool.from_function(
            func=multiply,
            name="Multiply Calculator",
            description="Use this to perform multiplications of two numbers. Use this tool by sending a pair of number separated by a comma.\nExample:1,2",
        ),
    ],
)

prompt = "Cost of $2.4 x $45.3 x $23.7"
# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

/var/folders/jm/6pbq_zts6k12d5j3w0c7pr6h0000gn/T/ipykernel_12550/3946161441.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.1)
/var/folders/jm/6pbq_zts6k12d5j3w0c7pr6h0000gn/T/ipykernel_12550/3946161441.py:13: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from



> Entering new AgentExecutor chain...
I need to multiply these three numbers together to find the total cost.
Action: Multiply Calculator
Action Input: 2.4,45.3
Observation: 108.71999999999998
Thought:I need to multiply the result by the third number.
Action: Multiply Calculator
Action Input: 108.72,23.7
Observation: 2576.6639999999998
Thought:I now know the final answer
Final Answer: $2576.66

> Finished chain.


{'input': 'Cost of $2.4 x $45.3 x $23.7', 'output': '$2576.66'}

#12.4 OpenAI Functions Agent

In [2]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")


class CalculatorTool(BaseTool):
    name: str = "CalculatorTool"
    description: str = """
    Use this to perform multiplications of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a * b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        CalculatorTool(),
    ],
)

prompt = "Cost of $2.4 x $45.3 x $23.7"
# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

/var/folders/jm/6pbq_zts6k12d5j3w0c7pr6h0000gn/T/ipykernel_14170/3833385283.py:28: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 2.4, 'b': 45.3}`


108.71999999999998
Invoking: `CalculatorTool` with `{'a': 108.72, 'b': 23.7}`


2576.6639999999998The cost of $2.4 x $45.3 x $23.7 is $2576.66.

> Finished chain.


{'input': 'Cost of $2.4 x $45.3 x $23.7',
 'output': 'The cost of $2.4 x $45.3 x $23.7 is $2576.66.'}

#12.5 Search Tool

In [ ]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain_community.tools import DuckDuckGoSearchResults

llm = ChatOpenAI(temperature=0.1)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name: str = "StockMarketSymbolSearchTool"
    description: str = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...


/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock'}`


snippet: 6. 防火墙或安全软件干扰：检查你的服务器或网络中是否存在防火墙或安全软件，它们可能会阻止 Cloudflare 对名称服务器的验证。 暂时禁用或调整相关设置， …, title: 我的cloudflare仍然无法验证我的名称服务器，怎么办？ - 知乎, link: https://www.zhihu.com/question/660688533, snippet: 小凳子认为，当 Cloudflare 的 域名解析 正常，但访问时遇到超时并报错522，这通常意味着尽管域名正确指向了服务器，但服务器与Cloudflare之间的连接存在 …, title: cloudflare域名解析正常，但访问超时并报错522，求解惑？ - 知乎, link: https://www.zhihu.com/question/667251550?write, snippet: 可以试试用 cloudflare zero trust 服务（即 Cloudflare Tunnel）。 域名访问 cloudflare --（隧道穿透）--你的服务器 由于 cloudflare 到你的服务器 …, title: 没有备案的域名怎么绕过国内服务器厂商的封锁？ - 知乎, link: https://www.zhihu.com/question/573163692?write, snippet: 是在今天的 2024 开发者周上，Cloudflare 发布了一系列令人激动的新特性，例如 Python Worker 以及 Workers AI，把应用开发与交付的便利性拔高到了一 …, title: 从技术上看，cloudflare比其他公司牛在哪儿？ - 知乎, link: https://www.zhihu.com/question/29886750

/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock ticker'}`
responded: I found information on Cloudflare, but I couldn't find the stock market ticker. Let me try again to find the ticker for Cloudflare's stock.

snippet: 6. 防火墙或安全软件干扰：检查你的服务器或网络中是否存在防火墙或安全软件，它们可能会阻止 Cloudflare 对名称服务器的验证。 暂时禁用或调整相关设置，然后再尝试验证。 7. 联系 …, title: 我的cloudflare仍然无法验证我的名称服务器，怎么办？ - 知乎, link: https://www.zhihu.com/question/660688533, snippet: 小凳子认为，当 Cloudflare 的 域名解析 正常，但访问时遇到超时并报错522，这通常意味着尽管域名正确指向了服务器，但服务器与Cloudflare之间的连接存在问题。, title: cloudflare域名解析正常，但访问超时并报错522，求解惑？ - 知乎, link: https://www.zhihu.com/question/667251550?write, snippet: 可以试试用 cloudflare zero trust 服务（即 Cloudflare Tunnel）。 域名访问 cloudflare --（隧道穿透）--你的服务器 由于 cloudflare 到你的服务器不依赖公网，直接绑定到你服务器上的任意端口， …, title: 没有备案的域名怎么绕过国内服务器厂商的封锁？ - 知乎, link: https://www.zhihu.com/question/573163692?write, snippet: 是在今天的 2024 开发者周上，Cloudflare 发布了一系列令人激动的新特性，例如 Python Worker 以及 Workers AI，把应用开发与交付的便利性拔高到了一个全新的程度。 与 Cloudflare 的 …, title: 从

/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what ticker does the stock have.",
 'output': "I apologize for the inconvenience, but I couldn't find the specific stock market ticker for Cloudflare's stock. However, I can still provide you with information on Cloudflare's stock performance and help you analyze if it's a potential good investment. Let's proceed with analyzing Cloudflare's stock without the ticker."}